In [1]:
from dsglib import getFile
import time

start = time.time()
userDict,labelNames = getFile('data/X_priv.csv')
stop = time.time()

print('import data time: '+str(stop-start))

import data time: 1.72733211517


In [2]:
# convert '' to -1 for IsPolicyholderAHomeowner
start=time.time()

IsPolicyholderAHomeownerIndex = labelNames.index('IsPolicyholderAHomeowner')
for user in userDict.values():
    for row in user:
        if(row[IsPolicyholderAHomeownerIndex]==''):
            row[IsPolicyholderAHomeownerIndex] = '-1'
stop=time.time()

print('Numerize time: '+str(stop-start))

Numerize time: 0.149775028229


In [3]:
import numpy as np

def stdv(values):
    return np.std(values)
def nu(values):
    return len(np.unique(values))
def isMostFrequent(values):
    un,ct = np.unique(values, return_counts=True)
    mostFrequent=un[np.argmax(ct)]
    return map(int,values==mostFrequent)

baseAugmentedFeatures = {
'CarAnnualMileage': stdv,
'CarUsageId': nu,
'CarInsuredValue': stdv,
'CarAge': stdv,
'FirstDriverDrivingLicenseNumberY': nu,
'VoluntaryExcess': stdv,
'CarParkingTypeId': nu, 
'PolicyHolderNoClaimDiscountYears': stdv,
'CoverIsNoClaimDiscountSelected': nu,
'CarDrivingEntitlement': nu,
'IsPolicyholderAHomeowner': nu,
'CarMakeId': nu,
'DaysSinceCarPurchase': stdv}

In [4]:
# convert featuresDict Names to Indexes
baseAugmentedFeaturesIndexes = {}
for featureName,featureFunc in baseAugmentedFeatures.items():
    featureIndex = labelNames.index(featureName)
    baseAugmentedFeaturesIndexes[featureIndex] = {'name': featureName, 'func': featureFunc}


In [5]:
# computeFeatures
start = time.time()

FirstDriverDrivingLicenseNumberYIndex = labelNames.index('FirstDriverDrivingLicenseNumberY')
CarMakeIdIndex = labelNames.index('CarMakeId')

newFeatures={}
for user in userDict.values():
    userNewFeatures = {}
    for featureIndex,featureInfo in baseAugmentedFeaturesIndexes.items():
        userFeatureArray = map(lambda row: float(row[featureIndex]), user)
        featureValue = featureInfo['func'](userFeatureArray)
        userNewFeatures['Variation'+featureInfo['name']] = featureValue
    # - FirstDriverDrivingLicenseNumberY/CarMakeId
    userNewFeatures['driverPerCar'] = float(userNewFeatures['VariationFirstDriverDrivingLicenseNumberY'])/userNewFeatures['VariationCarMakeId']
    # - FirstDriverDrivingLicenseNumberY/isMain
    FirstDriverDrivingLicenseNumberYMF = isMostFrequent(map(lambda row: float(row[FirstDriverDrivingLicenseNumberYIndex]), user))
    # - CarMakeId/isMain
    CarMakeIdMF = isMostFrequent(map(lambda row: float(row[CarMakeIdIndex]), user))
    for idx,row in enumerate(user):
        userNewFeatures['rowNumberForUser'] = len(user)
        userNewFeatures['isMostFrequentFirstDriverDrivingLicenseNumberY']=FirstDriverDrivingLicenseNumberYMF[idx]
        userNewFeatures['isMostFrequentCarMakeId']=CarMakeIdMF[idx]
        newFeatures[row[0]] = userNewFeatures

stop = time.time()
print('compute feature time: '+str(stop-start))

compute feature time: 12.3842520714


# Reorder and save

In [6]:
start=time.time()

featuresNameList = newFeatures['0'].keys()
with open('AugmentedFeaturesPriv.csv','w') as f:
    f.write(','+','.join(featuresNameList)+'\n')
    for idx in xrange(len(newFeatures)):
        f.write(str(idx)+',')
        orderedFeatureArray = []
        for featureName in featuresNameList:
            orderedFeatureArray.append(str(newFeatures[str(idx)][featureName]))
        f.write(','.join(orderedFeatureArray)+'\n')

stop=time.time()

print('save time: '+str(stop-start))

save time: 4.8407099247


# Features Stats

In [7]:
#import pandas as pd
#import pandas_profiling as pdpf
#features_set = pd.read_csv('AugmentedFeaturesTrain.csv')
#pdpf.ProfileReport(features_set)

In [8]:
#import pandas as pd
#import pandas_profiling as pdpf
#features_set = pd.read_csv('Y_test.predict')
#pdpf.ProfileReport(features_set)